In [1]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import os
import random
from tqdm import tqdm
from PIL import Image

import torchvision.transforms as transforms
import io
from torchvision.transforms.functional import to_pil_image, to_tensor


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(7)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
class ConditionalResize:
    def __init__(self, target_size):
        self.target_size = target_size

    def __call__(self, img):
        if max(img.size) < self.target_size:
            return transforms.Resize(self.target_size, interpolation=transforms.InterpolationMode.BILINEAR)(img)
        return img

In [5]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.df.iloc[idx, 0]
        image_path = f'/kaggle/input/image-classification-2024-spring/dataset/test/{image_name}'
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image

target_size = 380

test_transform = transforms.Compose([
    ConditionalResize(target_size),  # target_size보다 작은 경우에만 리사이즈
    transforms.CenterCrop(target_size),  # 모든 이미지를 target_size로 중앙 자르기
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
test_df = pd.read_csv('/kaggle/input/image-classification-2024-spring/submission_sample.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/image-classification-2024-spring/submission_sample.csv'

In [ ]:
test_dataset = CustomDataset(test_df, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torchvision.models as models
model = models.efficientnet_v2_s(pretrained=True)

num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

model = model.to(device)

# 모델 상태 불러오기
model_path = f'checkpoint/model5.pth'
model.load_state_dict(torch.load(model_path))
model.eval()
final_predictions = []

In [ ]:
with torch.no_grad():
    for images in tqdm(test_dataloader):
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs)
        _, predicted = torch.max(preds.data, 1)
        final_predictions += predicted.detach().cpu().numpy().tolist()

test_df['label'] = final_predictions

In [ ]:
test_df.to_csv('submission_model5.csv', index=False)